<!--start-block-->
<hr style="height: 1px;">
<i>This code was authored by the 8.316 Course Team, Copyright 2023 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

# LECTURE 17: Planetary Dynamics



<a name='section_17_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.0 Overview of Learning Objectives</h2>


Like what we saw for the pendulum, there are a broad range of techniques that can be used to numerically solve differential equations. Most numerical physics courses are taught by people who do a lot of this simulation, and as such like to dwell on the immense amount of work that has happened in the past 50 years.

A lot of this work is built on a huge amount of trial and error with the successes and failures having names after the people who did them. This makes it a bit hard to follow for someone like myself who doesn't know these people. However, much of this trial and error can be consolidated into some broad physics concepts that we can teach without trying and failing as many times. 

In this lecture we are going to build up the Hamiltonian Monte-Carlo Methods that are used for n-body  simulations. This part of this course wil just touch on the main elements. However, there is rich literature associated with this effort. We will dicuss this later. 


In [ ]:
import imageio
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize
import csv
from matplotlib.patches import Circle
from scipy.integrate import solve_ivp
#nbody
#https://courses.physics.ucsd.edu/2019/Winter/physics141/Lectures/Lecture14/renaud_thesis.pdf
#https://www.tat.physik.uni-tuebingen.de/~schaefer/teach/fum2020/f/nbody_slides.pdf
#https://github.com/bacook17/behalf/blob/master/behalf/octree.py
#https://anaroxanapop.github.io/behalf/#Code

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.


In [ ]:
#>>>RUN: L10.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_17_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.1 Simulating Planetary Motion</h2>  



This lecture will cover n-body simulations. The goal with n-body simulations is to be able to model stellar motion over many time periods. In practice, stellar motion boils down to just applications of Newton's laws, but on a large vareity of time and distance scales. 

The force for these two bodies is given by gravity, and we can write it as. 

$$
\vec{F} = \frac{G m_{1}m_{2}}{|\vec{r_{1}}-\vec{r_{2}}|^{3}} \left(\vec{r_{1}}-\vec{r_{2}}\right)
$$

where $G$ is the gravitation constant $m_{1,2}$ are the black hole masses, and  $\vec{q_{1}}$ and $\vec{q_{2}}$ are coordinates in space that describe the positions of the black holes. To describe the motion, we will take the Hamiltonian formalism. The reason is that the momenta $p_{1,2}$ and postitions $q_{1,2}$ follow Liouville's theorem, or in otherwords, we can write for a closed system without any energy in and out

$$
\frac{\partial\mathcal{H}}{\partial t} + \sum^{n}_{i=1} \left(\frac{\partial \mathcal{H}}{\partial q_{i}}\dot{q_{i}}+\frac{\partial 
\mathcal{H}}{\partial p_{i}}\dot{p_{i}}\right)=0 \\
\sum^{n}_{i=1}  \left( \frac{\partial \mathcal{H}} { \partial q_{i} } \dot{q_{i}}+\frac{\partial \mathcal{H}}{\partial p_{i}}\dot{p_{i}}\right) = 0
$$

where $\rho$ the density in momentum, position space is constant. Note that we can just get this by taking the time derivative (not just the partial) of the density $\rho(p,q)$ and propagating the full derivative.  

Let's consider a standard Hamiltonian for energy, given by 
$$
H = \frac{1}{2}\vec{p}^2 + \Phi(\vec{q})
$$
for a potential $\Phi$. Following the Hamiltonian formalism for motion, we can write Hamiltons equation as

$$
\frac{\partial H }{\partial p_{i}} =  \frac{d q_{i}}{dt} = \vec{p} \\
\frac{\partial H }{\partial q_{i}} = -\frac{d p_{i}}{dt} = -\nabla \Phi(\vec{q_{i}}) \\
$$
The simplest solutions involve straight up integration, we can write these as

$$
q_{\rm new} = \vec{q} + \Delta t \vec{p} \\
p_{\rm new} = \vec{p} - \Delta t \nabla \Phi(\vec{q_{i}}) \\
$$

Now from last class, we looked at the leap-frog Verlet stepping, which if you recall ensured that the detrminant of the updates are zero and that the stepping was volume preserving.

### Example of Sympletic approach

As a reminder from last time, if we consider a Hamiltonian given by the harmonic oscillator our updates become
$$
H = \frac{1}{2}\vec{p}^2+\frac{1}{2}\vec{q}^2 \\
q_{\rm new} = \vec{q} - \Delta t \vec{p} \\
p_{\rm new} = \vec{p} + \Delta t \vec{q} \\
H_{\rm new} = \frac{1}{2}\vec{p_{\rm new} }^2+\frac{1}{2}\vec{q_{\rm new} }^2 \\
H_{\rm new} = \frac{1}{2}(\vec{p} + \Delta t \vec{q})^2+\frac{1}{2}(q_{\rm new} = \vec{p} - \Delta t \vec{p})^2 \\
H_{\rm new} = \frac{1}{2}\vec{p}^2+\frac{1}{2}\vec{q}^2 + \Delta t \left(\vec{p}^2+\frac{1}{2}\vec{q}^2\right)
$$

Which for $\Delta t > 0$ clearly does not conserve energy.  Effectively every time we update $\Delta t$ 

Now if we look at our update and in the context of the Hamiltonian, we can make this "Symplectic" by making sure that our system conserves energy. In this scenario, we can do this in an elegant way by noting that for a 1 dimensional system all paths lie on a circle in phase space (ie $2H=C=p^2+x^2$ is the equation of circle) This means that all updates of momentum and position can be written as rotations 

$$
\begin{pmatrix}
x \\
p
\end{pmatrix}
=
\begin{pmatrix}
\cos \theta & \sin \theta \\
-\sin \theta & \cos \theta
\end{pmatrix}
\begin{pmatrix}
x \\
p
\end{pmatrix}
$$
And our updates above can be written 
$$
\begin{pmatrix}
x \\
p
\end{pmatrix}
=
\begin{pmatrix}
1 & \Delta t \\
-\Delta t & 1
\end{pmatrix}
\begin{pmatrix}
x \\
p
\end{pmatrix}
$$
Which has a determinant $1+\Delta t^2$ and is therefore not a rotation, (which have determinant 1) and the causes the phase space to increase. we can fix this by modifying our setup to correspond to a determinant of 1

$$
\begin{pmatrix}
x \\
p
\end{pmatrix}
=
\begin{pmatrix}
1 & \Delta t \\
-\Delta t & 1 -\Delta t^2
\end{pmatrix}
\begin{pmatrix}
x \\
p
\end{pmatrix}
$$
However, you might notice already, this is looks weird, its not really a rotation or a taylor expansion of a rotation. In fact, we can do better, but lets just go with this for a sec. If we write out the full Hamiltonian of this setup, we have 
$$
q_{\rm new} = \vec{p} - \Delta t \vec{p} \\
p_{\rm new} = (1-\Delta t^2) \vec{p} + \Delta t \vec{q} \\
H_{\rm new} = \frac{1}{2}\vec{p_{\rm new} }^2+\frac{1}{2}\vec{q_{\rm new} }^2 \\
H_{\rm new} = \frac{1}{2}\vec{p}^2+\frac{1}{2}\vec{q}^2 + \Delta t^2 \left(q^2-p^2\right) + 2\Delta t^3 qp + \Delta t^4 p^2
$$
which doesn't conserve the Hamiltonian, but because the determinant in phase space is 1, it is energy conserving, its just not for the Hamiltonian we care about. It turns out that its for 
$$
H_{\rm modified} = \frac{1}{2}\vec{p}^2+\frac{1}{2}\vec{q}^2  + \frac{\Delta t}{2} p q 
$$
This is crazy. What this means is that the timestep determines the conserved energy. This also means that we have to consider these elements when constructing our simulation if we want to make our simulation "simplectic" or energy preserving. There is no ideal solution to this. However, what this means is that we can make a "modified Hamiltoniain" denoted 
$$
H_{\rm modified} = H_{\rm true} + H_{\rm error}
$$
Importantly, if we change the time-step through a so-called "adaptive time step", we will break our conservation law in this scenario. 


### Back to gravity

Let's write this in the context of Gravity 

$$
H = \frac{\vec{p_{1}}^2}{2m_{1}} + \frac{\vec{p_{2}}^2}{2m_{2}} - \frac{Gm_{1}m_{2}}{|\vec{q_{1}}-\vec{q_{2}}|}
$$

This is a formula that we all know well. Despite it being two particles, we can write this in the center of mass frame as the motion of a single particle.  This gives us the constraint that 

$$
m_{1}\vec{r}_{1} + m_{2}\vec{r}_{2} = 0
$$

Let's go ahead and comput the force, and step it through, one thing we are going to do is modify our potential term slightly by

$$
H = \frac{\vec{p_{1}}^2}{2m_{1}} + \frac{\vec{p_{2}}^2}{2m_{2}} - \frac{Gm_{1}m_{2}}{|\vec{r_{1}}-\vec{r_{2}}+\epsilon|}
$$

This term $\epsilon$ is known as the softening term and it exists to avoid infinities. Let's go ahead and write everything out. The simplest verision of stepping we can do is 

$$
\vec{r_{1}} = \vec{r_{1}}+\vec{v_{1}}\Delta t \\
\vec{v_{1}} = \vec{v_{1}}+\Delta t \frac{G m_{2}}{|\vec{r_{1}}-\vec{r_{2}}+\epsilon|^{3}}\left(\vec{r_{1}}-\vec{r_{2}}\right) \\
\vec{r_{2}} = \vec{r_{2}}+\vec{v_{2}}\Delta t \\
\vec{v_{2}} = \vec{v_{2}}+\Delta t \frac{G m_{1}}{|\vec{r_{2}}-\vec{r_{1}}+\epsilon|^{3}}\left(\vec{r_{2}}-\vec{r_{1}}\right) \\
$$


Note that even though we did not write it above the $\epsilon$ softening term is still there helping us avoid the difficulties.  To step this, we are going to construct this with a python class. The strategy here is to be able to run this for an arbitrary amount of stars in the future siulations. 

As a result, we are going to simulate this with the leapfrog setup. 

With all simulations, we want to start with a basic check for what is going on. We can do this by just simulating circular motion. 

$$
F_{c} = \frac{mv^2}{r} = \frac{G Mm}{(2r)^2}\\
v     = \sqrt{\frac{GM}{4r}}
$$

In [ ]:
#Units
Gc=39.478 #AU^3/yr^2/Msun
re=1.0#AU
ve=2*np.pi*re#2pir/yr
Gmod=Gc/re**2

class star:
    #Save the history
    posh = np.array([])
    velh = np.array([])
    poth = np.array([])
    kinh = np.array([])
    soften =  1e-6
    
    def __init__(self,imass,xinit,yinit,vxinit,vyinit):
        self.mass = imass
        self.rpos = np.array([xinit,yinit])
        self.v    = np.array([vxinit,vyinit])
        self.a    = np.array([0.,0.])
        self.u    = 0
        
    def firststep(self,dt):
        self.rpos=self.rpos+0.5*dt*self.v         

    def firststepv(self,dt):
        self.v=self.v   +0.5*dt*self.a 
        self.rpos=self.rpos+dt*self.v
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
        
    def step(self,dt):
        #vnew = self.v   +dt*self.a 
        #self.rpos=self.rpos+dt*self.v
        #self.v   =vnew
        self.v    = self.v   +dt*self.a 
        self.rpos = self.rpos+dt*self.v
        self.posh = np.append(self.posh,self.rpos)
        self.velh = np.append(self.velh,self.v)
        self.poth = np.append(self.poth,self.u)
        self.kinh = np.append(self.kinh,0.5*self.mass*(np.dot(self.v,self.v)))
        #reset
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
        
    def force(self,istar):
        drv=istar.rpos-self.rpos
        drs=np.dot(drv,drv)+self.soften
        df=Gmod*drv*(drs**(-1.5))
        #if self.a[0] == 0: #something is f'd up
        #    self.a  = (istar.mass)*df
        #else:
        self.a  += (istar.mass)*df
        istar.update(df,self.mass)
        
    def update(self,df,imass):
        self.a += -imass*df    
        
    def update_u(self,iU):
        self.u += iU
        
    def potential(self,istar):
        drv=istar.rpos-self.rpos
        drs=(np.dot(drv,drv))**(-0.5)
        ulocal = -1.*0.5*Gmod*drs*self.mass*istar.mass
        self.u += ulocal
        istar.update_u(ulocal)
        
#now to get things going we are going to simulate a cricle
def circle_v(iM,iR):
    #F=mv^2/r = GMm/(2r)^2=>v=sqrt(GM/4r)
    return np.sqrt(Gc*iM/(4*iR))

def sim(dt=0.001,nsteps=10000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=star(mass, radius,0,0, vup)
    a2=star(mass,-radius,0,0,-vup)
    a1.firststepv(dt)
    a2.firststepv(dt)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))
    plt.plot(x1vals[:,0],x1vals[:,1])
    plt.plot(x2vals[:,0],x2vals[:,1])
    plt.show()

    plt.plot(range(nsteps),a1.poth+a2.poth,label='potential')
    plt.plot(range(nsteps),a1.kinh+a2.kinh,label='kinetic')
    plt.plot(range(nsteps),a1.kinh+a2.kinh+a1.poth+a2.poth,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()

sim(nsteps=10000)
sim(dt=0.25,nsteps=100)

It's worth noting that with the leap-frog integrator, there are still varations in th energy of the system.  We see this above with the coarse stepped version of the simulator. We can see this by zooming in on the energy regions

In [ ]:
def norm(iVal):
    return iVal/np.mean(iVal)

def simNormE():
    dt=0.001 #units of years
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=star(mass, radius,0,0, vup)
    a2=star(mass,-radius,0,0,-vup)
    nsteps=10000
    a1.firststepv(dt)
    a2.firststepv(dt)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)

    plt.plot(range(nsteps),norm(a1.poth+a2.poth),label='potential')
    plt.plot(range(nsteps),norm(a1.kinh+a2.kinh),label='kinetic')
    plt.plot(range(nsteps),norm(a1.kinh+a2.kinh+a1.poth+a2.poth),label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()

simNormE()

More importantly because the leap-frog step is simplectic(energy conserving) even though the stepping is not accurate, we have that a large number of iterations still preerves the overall energy of the system. This is really the most critical component of the leap-frog setup. As a result, it remains the current basis for how we do stellar simulations.  

Now for good measure, lets animate our setup. 

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def simNoPlot(dt=0.001,nsteps=10000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=star(mass, radius,0,0, vup)
    a2=star(mass,-radius,0,0,-vup)
    a1.firststepv(dt)
    a2.firststepv(dt)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))
    return np.array([x1vals,x2vals])


def makePlot(nbody,coords,ax,fig,images,ymin=-2,ymax=2,xmin=-2,xmax=2):
    # plot and show learning process
    plt.cla()
    ax.set_xlabel('x(AU)', fontsize=24)
    ax.set_ylabel('y(AU)', fontsize=24)
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    for body in range(nbody):
        #ax.plot(coords[body][-1,0],coords[body][-1,1],'o', color = '#d2eeff', markerfacecolor = '#0077BE')
        ax.plot(np.flip(coords[body][:,0]),np.flip(coords[body][:,1]), 'o-',color = '#d2eeff',markevery=10000, markerfacecolor = '#0077BE',lw=2)  
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
    

def animate(coords,iN=2,stepsize=50):
    images = []
    fig, ax = plt.subplots(figsize=(12,7))
    for step in range(len(coords[0])-110):
        if step % stepsize == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images)
    return images


from IPython.display import Image
xvals=simNoPlot()
images=animate(xvals)
imageio.mimsave('./orbit.gif', images, fps=10)
Image(open('orbit.gif','rb').read())


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-16.1.1 </span>

Repeat the above setup with the Euler step, what is going on? 

In [ ]:
class eulerstar:
    #Sve the history
    posh = np.array([])
    velh = np.array([])
    poth = np.array([])
    kinh = np.array([])
    soften =  1e-6
    
    def __init__(self,imass,xinit,yinit,vxinit,vyinit):
        self.mass = imass
        self.rpos = np.array([xinit,yinit])
        self.v    = np.array([vxinit,vyinit])
        self.a    = np.array([0.,0.])
        self.u    = 0
        
    def firststep(self,dt):
        self.rpos=self.rpos+0.5*dt*self.v         

    def firststepv(self,dt):
        self.v=self.v   +0.5*dt*self.a 
        self.rpos=self.rpos+dt*self.v
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
        
    def step(self,dt):
        vold=self.v
        self.v    = self.v   +dt*self.a 
        self.rpos = self.rpos+dt*vold
        self.posh = np.append(self.posh,self.rpos)
        self.velh = np.append(self.velh,self.v)
        self.poth = np.append(self.poth,self.u)
        self.kinh = np.append(self.kinh,0.5*self.mass*(np.dot(self.v,self.v)))
        #reset
        self.a[0] = 0.
        self.a[1] = 0.
        self.u    = 0.
        
    def force(self,istar):
        drv=istar.rpos-self.rpos
        drs=np.dot(drv,drv)+self.soften
        df=Gmod*drv*(drs**(-1.5))
        #if self.a[0] == 0: #something is f'd up
        #    self.a  = (istar.mass)*df
        #else:
        self.a  += (istar.mass)*df
        istar.update(df,self.mass)
        
    def update(self,df,imass):
        self.a += -imass*df    
        
    def update_u(self,iU):
        self.u += iU
        
    def potential(self,istar):
        drv=istar.rpos-self.rpos
        drs=(np.dot(drv,drv))**(-0.5)
        ulocal = -1.*0.5*Gmod*drs*self.mass*istar.mass
        self.u += ulocal
        istar.update_u(ulocal)
        
#now to get things going we are going to simulate a cricle
def circle_v(iM,iR):
    #F=mv^2/r = GMm/(2r)^2=>v=sqrt(GM/4r)
    return np.sqrt(Gc*iM/(4*iR))

def eulersim(dt=0.001,nsteps=10000):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    a1=eulerstar(mass, radius,0,0, vup)
    a2=eulerstar(mass,-radius,0,0,-vup)
    for t in range(nsteps):
        a1.force(a2)
        a1.potential(a2)
        a1.step(dt)
        a2.step(dt)
    x1vals=np.reshape(a1.posh,(len(a1.posh)//2,2))
    x2vals=np.reshape(a2.posh,(len(a2.posh)//2,2))
    plt.plot(x1vals[:,0],x1vals[:,1])
    plt.plot(x2vals[:,0],x2vals[:,1])
    plt.show()

    plt.plot(range(nsteps),a1.poth+a2.poth,label='potential')
    plt.plot(range(nsteps),a1.kinh+a2.kinh,label='kinetic')
    plt.plot(range(nsteps),a1.kinh+a2.kinh+a1.poth+a2.poth,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()

eulersim()
eulersim(dt=0.25,nsteps=100)

This shouldn't come as a surprise to anybody as this point. The Euler step does not conserve energy and so our orbit will slowly fall apart. The more we step it out, the more things get worse. 

<a name='section_16_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L16.2 Parallel Dynamics </h2>  


Now, given this  start, we would like to have some fun to explore the nature of this simulation to create interesting dynamics. With 2-bodies things are kind of trivial. However, when we go to 3-bodies, we suddenly have a very rich and interesting system, that we can explore. 

However before, we go there, lets take some time to speed up our simulation so that we can really get the best out of our computations. 

The best way to speed up the computation is to : 

 * Use the numpy libraries as much as possible
 * Parallelize the computation
 
To make this fast, what we will do is store all the stars in an array, and do the computation of Newton's lawas in parallel. As a result, lets make a class called "stars". 

The way we will compute the force is we will take a vector of x-positions, a vector of y-positions and comput teh diferrences. To be clear let us define 

$$
\vec{x} = x_{i}\hat{i} \\
\vec{y} = y_{i}\hat{i} \\
$$

or the i-th element of the vector is the x and y position.  As a result, the difference in radius is defined as

$$
dx_{ij} = x_{j} - x_{i} = \vec{x}^{T}-\vec{x}\\
dr_{ij} = \sqrt{dx_{ij}^2 + dy_{ij}^2}
$$

here we hae computed a matrix $i$-$j$. We can then contract down this matrix by multiplying it by the respective coordinates. We can write this as:

$$
a^{i}_{x} = G m_{j} {dr_{ij}}^{-3} dx_{ij} \\
$$
To compress along an axis, we have to use the @ symboly 

Let's quickly play with some arrays so that we know whats going on. 


In [ ]:
x = np.array([[1,2,3]])
y = np.array([[3,4,5]])
print("Vecs:\n",x.T,x)
print("Matrix\n",x.T-x)
print("Matrix\n",y.T-y)

dr2 = (x.T-x)**2 + (y.T-y)**2
print("Matrix dr:\n",dr2)
mass = np.array([1,2,3])
print("x*m",x * mass)
print("x@m",x @ mass)
print("Matrix * m \n",(x.T-x) @ mass)


Now the construction above is great for creating the so called adjacency matrix. The adjancency matrix for distance can be written as 

$$
r_{ij} = \sqrt{\Delta x_{ij}^2 + \Delta y_{ij}^2}
$$

which has a distance for $i$ to $j$ in the i-th,j-th element, and the same distance in the j-th,i-th element. We can sum over the distances by just taking the upper triangle. We can do this through the triu  function of numpy. Let's take a quick look at how it works.  

In [ ]:
print("Matrix dr:\n",dr2)
print("Tri dr:\n",np.triu(dr2))
print(" Sum Tri dr:\n",np.sum(np.triu(dr2)))


Alright, lets go ahead and code up everything into a new class, we call Stars. Take a look at the energy and acceleration computations, we are aiming to comput them all a the same time. 

In [ ]:
class stars:
    test=1
    soften = 1e-6
    posh = np.array([])
    velh = np.array([])
    toth = np.array([])
    
    def __init__(self,imass,pos,vpos,n):
        self.mass = imass
        self.rpos = pos
        self.v    = vpos
        self.e    = 0
        self.n    = n
    
    def parallelAcc(self): #take in arrays of everything
        xpos=self.rpos[:,0:1]
        ypos=self.rpos[:,1:2]
        dx = xpos.T - xpos
        dy = ypos.T - ypos
        dr2  = dx**2 + dy**2 + self.soften**2
        dr2[dr2>0] = dr2[dr2>0]**(-1.5)
        #idr3 = (dr2**(-1.5))
        ax   = Gmod * (dx*dr2) @ self.mass
        ay   = Gmod * (dy*dr2) @ self.mass
        a = np.vstack((ax,ay))
        self.a = a.T
    
    def totalE(self):
        xpos=self.rpos[:,0:1]
        ypos=self.rpos[:,1:2]
        dx = xpos.T - xpos
        dy = ypos.T - ypos
        dr = dx**2 + dy**2
        dr = np.sqrt(dx**2 + dy**2)
        dr[dr > 0] = 1./dr[dr > 0]
        idr = -Gmod*self.mass.T*(self.mass*dr)
        totalU = np.sum(np.sum(np.triu(idr)))
        totalK = np.sum(0.5*self.mass*np.sum(self.v**2,1))
        self.e = totalK+totalU
        
    def firststep(self,dt):
        self.v    = self.v   +0.5*dt*self.a        
        self.rpos = self.rpos+dt*self.v
        
    def step(self,dt):
        self.v    = self.v   +dt*self.a 
        self.rpos = self.rpos+dt*self.v
        self.posh = np.append(self.posh,self.rpos)
        self.velh = np.append(self.velh,self.v)
        self.toth = np.append(self.toth,self.e)

#Now lets setup a quick init script only for up to 4 stars 
def initparts(iN):
    radius=1#units of AU
    mass=1#units of Solar Mass
    vup=circle_v(mass,radius)
    mass = np.ones(iN)*mass # constant mass for now
    #position
    pos=np.array([])
    pos  = np.append(pos,np.array([radius,0]))
    pos  = np.append(pos,np.array([-radius,0]))
    if iN > 2:
        pos  = np.append(pos,np.array([0,radius]))
    if iN > 3:
        pos  = np.append(pos,np.array([0,-radius]))
    #velocity
    vel=np.array([])
    vel  = np.append(vel,np.array([0,vup]))
    vel  = np.append(vel,np.array([0,-vup]))
    if iN > 2:
        vel  = np.append(vel,np.array([-vup,0]))
    if iN > 3:
        vel  = np.append(vel,np.array([ vup,0]))
    pos  = np.reshape(pos,(iN,2))
    vel  = np.reshape(vel,(iN,2))
    return pos,vel,mass
        
def plotPaths(iN,xvals,evals):
    x1vals=np.reshape(xvals,(len(evals),2*iN))
    for i0 in np.arange(iN):
        plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])
    plt.show()

    plt.plot(range(len(evals)),evals,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()


def simN2(iN=2,insteps=5):
    dt=0.001 #units of years
    pos,vel,mass=initparts(iN) 
    allstars = stars(mass,pos,vel,iN)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    
    plotPaths(iN,allstars.posh,allstars.toth)

simN2(2,insteps=10000)

Now let's setup a random init that allows for an abitrary number of stars. For this we can randomly sample over a Gaussian distribution. 

In [ ]:
#Now lets setup a quick init script only for up to 4 stars 
def initparts(iN):
    ###Lets simplify the coordinates
    ###sample a radius along the x-axis 
    #radius=np.ones(iN)
    #radius[0]=-1    
    radius=np.random.uniform(0.5,3,iN)#units of AU
    radius[1]=-radius[0]
    theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta=0
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    mass=np.random.uniform(0,1,iN)#units of Solar Mass
    #mass=np.ones(iN)#units of Solar Mass
    #now v
    #v=circle_v(mass,np.abs(radius))*np.random.normal(0,3)
    v=circle_v(mass,np.abs(radius))*np.abs(np.random.normal(0,0.1))
    theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass



def simN2(iN=2,insteps=5):
    dt=0.001 #units of years
    pos,vel,mass=initparts(iN) 
    allstars = stars(mass,pos,vel,iN)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

def animate(coords,iN=2):
    images = []
    fig, ax = plt.subplots(figsize=(12,7))
    print(len(coords[0]),coords.shape)
    for step in range(len(coords[0])-110):
        if step % 50 == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=-10,ymax=10,xmin=-10,xmax=10)
    return images

np.random.seed(109)
xvals=simN2(4,insteps=10000)
images=animate(xvals,iN=4)
imageio.mimsave('./orbit_4.gif', images, fps=10)
Image(open('orbit_4.gif','rb').read())


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-16.1.1 </span>

Repeat the above setup for 100 stars using taking 1000 steps for the parallel and the old approach. Can you see how much faster this is? 

In [ ]:
import itertools

def simN(iN,dt=0.001,insteps=1000):
    combos=list(itertools.combinations(np.arange(iN), 2))
    pos,vel,mass=initparts(iN) 
    stars=np.array([])
    for pN in range(iN):
        a = star(mass[pN],pos[pN,0],pos[pN,1],vel[pN,0],vel[pN,1])
        stars = np.append(stars,a)
    for combo in combos:
        stars[combo[0]].force(stars[combo[1]])
    for pStar in stars:
        pStar.firststepv(dt)
    for t in range(insteps):
        for combo in combos:
            stars[combo[0]].force(stars[combo[1]])
            stars[combo[0]].potential(stars[combo[1]])
        for pStar in stars:
            pStar.step(dt)

    totalE=np.zeros(insteps)
    for i0 in np.arange(iN):
        x1vals=np.reshape(stars[i0].posh,(len(stars[i0].posh)//2,2))
        plt.plot(x1vals[:,0],x1vals[:,1])
        totalE += (stars[i0].poth+stars[i0].kinh)
    plt.show()
    plt.plot(range(insteps),totalE,label='Total')
    plt.legend()
    plt.xlabel('time step')
    plt.ylabel('energy')
    plt.show()
    return 

import time
print("Start Basic:")
start_time = time.time()
xvals=simN(100,insteps=100)
print("Stop Basic --- %s seconds ---" % (time.time() - start_time))

print("Start Parallel")
start_time = time.time()
xvals=simN2(100,insteps=100)
print("Stop Parallel --- %s seconds ---" % (time.time() - start_time))


<a name='section_16_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L16.3 3-body Problem </h2>  


Now that we have a setup that works well, lets take some time to explore the 3-body problem. Here, we can look at the dynamics and have a little fun, to see how motion works with 3-bodies involved. Let's consider a few 3-body situations and look at the motion we have with our numrical simulation. 

Its fun to play with these solutions since there is a wide variety of solution that are possible. 

In [ ]:
#Now lets setup a quick init script only for up to 4 stars 
def init3body(iN):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*1.5
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass



def simN2(iN=3,insteps=5):
    dt=0.001 #units of years
    pos,vel,mass=init3body(iN)
    allstars = stars(mass,pos,vel,iN)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    print(x1vals.shape)
    return x1vals

def animate(coords,iN=2):
    images = []
    fig, ax = plt.subplots(figsize=(12,7))
    print(len(coords[0]),coords.shape)
    for step in range(len(coords[0])-110):
        if step % 50 == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=-3,ymax=3,xmin=-3,xmax=3)
    return images


xvals=simN2(3,insteps=10000)
images=animate(xvals,iN=3)
imageio.mimsave('./orbit_3.gif', images, fps=10)
Image(open('orbit_3.gif','rb').read())


In [ ]:
#Now lets setup a quick init script only for up to 4 stars 
def init3body(iN):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*1.3
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass


#np.random.seed(300)
xvals=simN2(3,insteps=10000)
images=animate(xvals,iN=3)
imageio.mimsave('./orbit_3_v1.gif', images, fps=10)
Image(open('orbit_3_v1.gif','rb').read())


In [ ]:
#Now lets setup a quick init script only for up to 4 stars 
def init3body(iN):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*2.0
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass

def animate(coords,iN=2,ymin=-10,ymax=10,xmin=-10,xmax=10,stepsize=50):
    images = []
    fig, ax = plt.subplots(figsize=(12,7))
    for step in range(len(coords[0])-110):
        if step % stepsize == 0:
            makePlot(iN,coords[:,step:step+100],ax,fig,images,ymin=ymin,ymax=ymax,xmin=xmin,xmax=xmax)
    return images

xvals=simN2(3,insteps=10000)
images=animate(xvals,iN=3)
imageio.mimsave('./orbit_3_v2.gif', images, fps=10)
Image(open('orbit_3_v2.gif','rb').read())

In [ ]:
#Now lets setup a quick init script only for up to 4 stars 
def init3body(iN,scale):
    radius = np.ones(iN)
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    #now v
    v=circle_v(mass,np.abs(radius))*scale
    theta+=np.pi/2.
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,v,mass

def plotJustPaths(iN,xvals,evals):
    x1vals=np.reshape(xvals,(len(evals),2*iN))
    for i0 in np.arange(iN):
        plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])


def simInit(iN=3,insteps=5,scale=1.0):
    dt=0.001 #units of years
    pos,vel,mass=init3body(iN,scale)
    allstars = stars(mass,pos,vel,iN)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotJustPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

for scale in np.arange(0.5,2.5,0.1):
    x1vals=simInit(3,5000,scale) 

plt.xlim(-10,10)
plt.ylim(-10,10)
plt.xlabel("x[AU]")
plt.ylabel("x[AU]")
plt.show()

Now for fun, lets consider a system like the earth and sun setup, and we can try to put a light planet aorund and solve for the motion in a vareity of forms. 

In [ ]:
#Now lets setup a quick init script only for up to 4 stars 
def init3body(iN,scale):
    radius = np.ones(iN)
    radius[0] = 0.0001
    #radius=np.random.uniform(0.5,3,iN)#units of AU
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta = np.arange(0,2*np.pi,2*np.pi/iN)
    theta[0] = 0
    theta[1] = 0
    theta[2] = np.pi*0.5
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    #mass=np.random.uniform(0,1,iN)#units of Solar Mass
    mass=np.ones(iN)#units of Solar Mass
    mass[1] *= 0.01
    mass[2] *= 0.001
    #now v
    mr = np.ones(iN)
    v=circle_v(mr,np.abs(radius*0.25))
    theta+=np.pi/2.
    v[0] = 0
    #v=np.zeros(iN)
    #theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    print(v)
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    print(vcom)
    return pos,v,mass

def plotJustPaths(iN,xvals,evals):
    x1vals=np.reshape(xvals,(len(evals),2*iN))
    for i0 in np.arange(iN):
        plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])


def simInit(iN=3,insteps=5,scale=1.0):
    dt=0.001 #units of years
    pos,vel,mass=init3body(iN,scale)
    allstars = stars(mass,pos,vel,iN)
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    plotJustPaths(iN,allstars.posh,allstars.toth)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

x1vals=simInit(3,5000,1) 

plt.xlim(-2,2)
plt.ylim(-2,2)
plt.xlabel("x[AU]")
plt.ylabel("x[AU]")
plt.show()

images=animate(x1vals,iN=3,ymin=-2,ymax=2,xmin=-2,xmax=2)
imageio.mimsave('./orbit_3_v3.gif', images, fps=10)
Image(open('orbit_3_v3.gif','rb').read())

<a name='section_16_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L16.4 N-body Problem </h2>  


Now, we have been able to scale our simulation so that it can work pretty well on $mathcal{O}(100)$ different planets. However, we would really to make a setup that can capture the dynamics for 1000s of bodies of stars. Something that we could potentially use to model our galaxy? 

By going to larger and larger numebers of stars, we can start to capture the full dynamics present when we perform large scale astronomical simulations. The problem with going to large and large number of stars is that we know have more and more pairwise computations to compute the force.  Simply put the force on a single body is : 

$$
\vec{a}_{i} = \sum_{j} \frac{G M_{j}}{|\vec{r}_{j}-\vec{r}_{i}|^{3}}\left(\vec{r}_{i}-\vec{r}_{j}\right)
$$

Which requires a sum over $j \in N_{\rm body}$.  Computing this for all values $i\in N_{\rm body}$ gives us a computation that scales as $\mathcal{N_{\rm body}^2}$. Which means that for $N_{\rm body}=1000$, we are talking about a computation that is $\mathcal{O}(10^{6})$ in computations. 

Now an important point that should be made with large scale stellar simulations is the notion of timescale. We we have two masses that are very far apart, the amount of updates that need to happen is much smaller than if we have two masses that are nearby. The point being that stars that all have roughly the velocity the time updates will impact the local dynamics when $r$ is small compared to large $r$. We can then write a time update as : 

$$
\Delta t \propto \frac{\Delta v_{ij}}{\Delta r_{ij}} \\
\Delta t \propto \frac{1}{\Delta r_{ij}}
$$

and in practice the velocities are typically very simlar on large scales, so this yields us to the second equation. Now conceptually, we can also think of the length scale as a way to resolve various tiers of resolution in n-body computations. What I mean is that we can imagine instead of computing the individual forces of many stars that are far away, we can average the mass and the distances and following Gauss's law treat these as a single particle with a combined mass given by the sum of the stars. 

Pracitally speaking this makes our summation over the points given by 

$$
\vec{a}_{i} = \sum_{j \in \Delta r < \epsilon} \frac{G M_{j}}{|\vec{r}_{j}-\vec{r}_{i}|^{3}}\left(\vec{r}_{i}-\vec{r}_{j}\right) + \frac{G \sum_{j \in r > \epsilon} m_{j}} { |\vec{r}_{i}-\sum_{j \in r > \epsilon} m_{j}\vec{r_{j}}|^{3}} \left(\vec{r}_{i}-\sum_{j \in r > \epsilon} m_{j}\vec{r_{j}}\right)
$$

In other words for distances larger than some value $\epsilon$ we just sum over all the stars and compute the average distances to that point. 

Now to dermine a reasonable average, the way we approach this is by building a tree structure, this is often done by building a [k-d tree](https://en.wikipedia.org/wiki/K-d_treek-d).  This tree will take an image, subdivide into two subimages, then take each sub image and subdivide it and so on. Lets take a look at how it looks. 

Before we go the whole way, lets just play with a 1D dataset to see how it works

In [ ]:
xarr = np.random.random((10, 1)) * 2 - 1
print(xarr)
print("Max:",np.argmax(xarr))
print("Sort:",np.argsort(xarr,axis=0))
xarr=xarr[np.argsort(xarr,axis=0)]
print("half:",xarr[xarr.shape[0]//2])

From this, we can start to take an array and divide it into subsets. by splitting the x or y position of the samples. In light of this, lets make a KDTree that starts to cut the data into two using either the y or the x-axis. We will make this tree recursive, so that we recurse lower and lower within the tree. 

In [ ]:
class KDTree:
    """Simple KD tree class"""

    # class initialization function
    def __init__(self, data, mins, maxs):
        self.data = np.asarray(data)

        # data should be two-dimensional
        assert self.data.shape[1] == 2

        if mins is None:
            mins = data.min(0)
        if maxs is None:
            maxs = data.max(0)

        self.mins  = np.asarray(mins)
        self.maxs  = np.asarray(maxs)
        self.sizes = self.maxs - self.mins

        self.child1 = None
        self.child2 = None

        if len(data) > 1:
            # sort on the dimension with the largest spread
            largest_dim  = np.argmax(self.sizes)
            i_sort       = np.argsort(self.data[:, largest_dim])
            self.data[:] = self.data[i_sort, :]

            # find split point
            N = self.data.shape[0]
            half_N = int(N / 2)
            split_point = 0.5 * (self.data[half_N, largest_dim] + self.data[half_N - 1, largest_dim])

            # create subnodes
            mins1 = self.mins.copy()
            mins1[largest_dim] = split_point
            maxs2 = self.maxs.copy()
            maxs2[largest_dim] = split_point

            # Recursively build a KD-tree on each sub-node
            self.child1 = KDTree(self.data[half_N:], mins1, self.maxs)
            self.child2 = KDTree(self.data[:half_N], self.mins, maxs2)

    def draw_rectangle(self, ax, depth=None):
        """Recursively plot a visualization of the KD tree region"""
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.sizes, ec='k', fc='none')
            ax.add_patch(rect)

        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)

    
#------------------------------------------------------------
# Create a set of structured random points in two dimensions
np.random.seed(0)

X = np.random.random((30, 2)) * 2 - 1
X[:, 1] *= 0.1
X[:, 1] += X[:, 0] ** 2 #y=x^2

#------------------------------------------------------------
# Use our KD Tree class to recursively divide the space
KDT = KDTree(X, [-1.1, -0.1], [1.1, 1.1])

#------------------------------------------------------------
# Plot four different levels of the KD tree
fig = plt.figure(figsize=(10, 10))
fig.subplots_adjust(wspace=0.1, hspace=0.15,left=0.1, right=0.9, bottom=0.05, top=0.9)

for level in range(1, 5):
    ax = fig.add_subplot(2, 2, level)#, xticks=[], yticks=[])
    ax.scatter(X[:, 0], X[:, 1], s=9)
    KDT.draw_rectangle(ax, depth=level - 1)

    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-0.15, 1.15)
    ax.set_title('level %i' % level)

# suptitle() adds a title to the entire figure
fig.suptitle('$k$d-tree Example')
plt.show()

Now for a specific point. What we can now do is use this tree structure to merge elements into a single object. The way we will do this is to split stars. 

The problem with teh above is that while this tree splits the data by the density of the points, it requires that we sort the dataset so that we can split it into smaller datasets. Do you understand why this could be a problem? 
​
Sort requires pairwise comparisons. This is a $\mathcal{O}(n^{2})$, and it defeats the point of trying to avoid the large $N$ set of comparisons. Hence, we can't actually sort along this direction, we need to just geometrically divide this space using the full range of the system.  

In [ ]:
class SquareTree:
    """KD Tree aimed"""

    # class initialization function
    def __init__(self,iMins,iMaxs,iAxis=0,idepth=0):
        self.m     = 0
        self.depth = idepth+1
        self.mins=iMins
        self.maxs=iMaxs
        self.dxs = self.maxs - self.mins
        self.mass   = 0
        self.posm   = np.array([0,0])
        self.pos    = np.array([0,0])
        self.child1 = None
        self.child2 = None
        minsR = self.mins.copy()
        maxsL = self.maxs.copy()
        #iterative funciton that defines the splitting
        if self.depth > 6: #6 levels deep
            return
        oAxis=1 #alternate x and y-axis (don't need to BTW)
        if iAxis == 1:
            oAxis = 0
        maxsL[iAxis]=iMaxs[iAxis]-self.dxs[iAxis]*0.5 #Now split the range between left and right in physical half
        minsR[iAxis]=iMins[iAxis]+self.dxs[iAxis]*0.5 # li
        self.child1 = SquareTree(self.mins,maxsL,oAxis,self.depth)
        self.child2 = SquareTree(minsR,self.maxs,oAxis,self.depth)
            
    def draw_rectangle(self, ax, depth=None):
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.dxs, ec='k', fc='none')
            ax.add_patch(rect)

        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)


#now lets make a star object that stpes 
class body():
    def __init__(self, ipos, ivpos, imass):
        self.rpos = ipos
        self.v    = ivpos
        self.mass = imass
        self.a    = np.array([0,0])
        
    def firststep(self,dt):
        self.v=self.v   +0.5*dt*self.a 
        self.rpos=self.rpos+dt*self.v
        
    def step(self,dt):
        #vnew = self.v   +dt*self.a 
        #self.rpos=self.rpos+dt*self.v
        #self.v   =vnew
        self.v    = self.v   +dt*self.a 
        self.rpos = self.rpos+dt*self.v

        
def grid(iX):
    Xmin = np.min(iX,axis=0)
    Xmax = np.max(iX,axis=0)
    SQT  = SquareTree(Xmin,Xmax,0)
    return SQT

def points(iBodies):
    lXs = np.array([])
    for pBody in iBodies:
        lXs = np.append(lXs,pBody.rpos)
    lXs = np.reshape(lXs,(len(iBodies),2))
    return lXs


#First we will sample 3000 stars in 2D space with 2D velocity
np.random.seed(1234)
X = np.random.random((30, 2)) * 2 - 1
V = np.random.random((30, 2)) * 0.1 
#X[:, 1] *= 0.1
#X[:, 1] += X[:, 0] ** 2
bodies = []
for i0 in range(len(X)):
    mass=1. # they will all have a mass of a solar mass
    pBody = body(X[i0],V[i0],mass)
    bodies.append(pBody)

lXs = points(bodies)
SQT = grid(lXs)
fig1 = plt.figure(figsize=(10, 10))
fig1.subplots_adjust(wspace=0.1, hspace=0.15,left=0.1, right=0.9,bottom=0.05, top=0.9)
for level in range(1, 6):
    ax = fig1.add_subplot(2, 3, level)
    ax.scatter(lXs[:, 0], lXs[:, 1], s=9)
    SQT.draw_rectangle(ax, depth=level)
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title('level %i' % level)
plt.show()


Once we have this structure, what we can do is fill it with all of our objects, and then we can compute the force by constructing a force law that relies on grid values for things that are far away and positions for things that are close. 

For this, our we can define our force law as: 

$$
\vec{a_{i}} = G\frac{\sum_{j\in grid} m_{j}} {|\vec{r}_{i}-\frac{1}{m_{\rm tot}}\sum_{j\in grid} m_{j}\vec{r}_{j}|^{3}} \left(\vec{r_{i}}-\frac{1}{m_{\rm tot}}\sum_{j\in grid} \vec{m_{j} r_{j}}\right)
$$

What this means is that when we populate the grid, we need to compute two things

$$
m_{\rm tot}^{grid}=\sum_{j\in grid} m_{j} \\
r_{\rm tot}^{grid}=\frac{1}{m_{\rm tot}^{grid}} \sum_{j\in grid} m_{j}\vec{r_{j}} \\
$$

This allows us to define two new functions to our tree that keep track of the mass and teh acceleration after, we ahve defined teh grid.  

In [ ]:
class SquareTree:
    """KD Tree aimed"""

    # class initialization function
    def __init__(self,iMins,iMaxs,iAxis=0,idepth=0):
        self.m      = 0
        self.depth  = idepth+1
        self.mins   = iMins
        self.maxs   = iMaxs
        self.dxs    = self.maxs - self.mins
        self.mass   = 0
        self.posm   = np.array([0.,0.])
        self.pos    = np.array([0.,0.])
        self.maxdepth=6
        self.child1 = None
        self.child2 = None
        minsR = self.mins.copy()
        maxsL = self.maxs.copy()
        #iterative funciton that defines the splitting
        if self.depth > self.maxdepth: #6 levels deep
            return
        oAxis=1 #alternate x and y-axis (don't need to BTW)
        if iAxis == 1:
            oAxis = 0
        maxsL[iAxis]=iMaxs[iAxis]-self.dxs[iAxis]*0.5 #Now split the range between left and right in physical half
        minsR[iAxis]=iMins[iAxis]+self.dxs[iAxis]*0.5 # li
        self.child1 = SquareTree(self.mins,maxsL,oAxis,self.depth)
        self.child2 = SquareTree(minsR,self.maxs,oAxis,self.depth)
            
    def addmass(self,mass,pos):
        #compute for whole grid
        self.m    += mass
        self.posm += mass*pos #intermediate value
        self.pos   = self.posm/self.m #mass weighted avg
        #compute it for the children
        if self.child1 == None:
            return 
        if pos[0] <= self.child1.maxs[0] and pos[1] <= self.child1.maxs[1]:
            self.child1.addmass(mass,pos)
        else:
            self.child2.addmass(mass,pos)
    
    def accel(self,pos):
        #compute it for the grid
        if self.m == 0:
            return 0
        #compute the fine grained resolution if distance > x
        if self.dist(pos)/np.max(self.dxs) < 4 and self.depth < self.maxdepth:
            a1 = self.child1.accel(pos)
            a2 = self.child2.accel(pos)
            return a1+a2
        #else compute the coarse grain guy
        dx  = self.pos-pos
        dx3 = (np.dot(dx,dx)+1e-6)**(-1.5)
        return Gmod*self.m*dx*dx3
    
    def dist(self,pos):
        #distance to grid center
        dx  = self.pos-pos
        return np.dot(dx,dx)
    
    def draw_rectangle(self, ax, depth=None):
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.dxs, ec='k', fc='none')
            ax.add_patch(rect)

        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)


class TreeStars:
    def __init__(self,iBodies,iX,iAxis=0,idepth=0):
        Xmin = np.min(iX,axis=0)
        Xmax = np.max(iX,axis=0)
        self.posh   = np.array([])
        self.grid   = SquareTree(Xmin,Xmax,0)
        self.bodies = iBodies
        self.nsteps = 0
        
    def addMass(self):
        for pBody in self.bodies:
            self.grid.addmass(pBody.mass,pBody.rpos)

    def force(self):
        for pBody in self.bodies:
            pBody.a = self.grid.accel(pBody.rpos)

    def firststep(self,dt):
        for pBody in self.bodies:
            pBody.firststep(dt)

    def step(self,dt):
        for pBody in self.bodies:
            pBody.step(dt)
        
    def points(self):
        lXs = np.array([])
        for pBody in self.bodies:
            lXs = np.append(lXs,pBody.rpos)
        lXs = np.reshape(lXs,(len(self.bodies),2))
        return lXs
    
    def store(self):
        lXs = self.points()
        self.posh = np.append(self.posh,lXs)
    
    def history(self):
        return np.reshape(self.posh,(self.nsteps,len(self.bodies),2))
    
    def regrid(self):
        lX = self.points()
        Xmin = np.min(lX,axis=0)
        Xmax = np.max(lX,axis=0)
        self.grid  = SquareTree(Xmin,Xmax,0)
        self.addMass()

    def allsteps(self,insteps=5000,dt=0.001):
        nsteps=insteps
        self.nsteps+=nsteps
        self.regrid()
        self.firststep(dt)
        self.regrid()
        for t in range(nsteps):
            self.force()
            self.step(dt)
            self.regrid()
            self.store()
        return self.points()

treeStar = TreeStars(bodies,X)
lX=treeStar.allsteps()
fig1 = plt.figure(figsize=(10, 10))
#fig1.subplots_adjust(wspace=0.1, hspace=0.15,left=0.1, right=0.9,bottom=0.05, top=0.9)
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(lX[:, 0], lX[:, 1])
#ax.set_xlim(-1.2, 1.2)
#ax.set_ylim(-0.15, 1.15)
plt.show()

tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
images=animate(tracks,iN=30,xmin=-500,xmax=500,ymin=-500,ymax=500)
imageio.mimsave('./orbit_n_v0.gif', images, fps=10)
Image(open('orbit_n_v0.gif','rb').read())

This looks really cool, but is this doing what we expect? Lets do a quick test with a two body system to make sure everything makes sense. 

In [ ]:
bodies = []
mass=1 # they will all have a mass of a solar mass
X = np.array([1,0])
X = np.vstack((X, np.array([-1,0]) ))
vc=circle_v(mass,1)
V = np.array([0,-vc])
V = np.vstack((V, np.array([0,vc]) ))
#Xr = np.array([[4,4],[-4,-4]])

bodies=[]
pBody = body(X[0],V[0],mass)
bodies.append(pBody)
pBody = body(X[1],V[1],mass)
bodies.append(pBody)

treeStar = TreeStars(bodies,X)
lX=treeStar.allsteps(insteps=1000,dt=0.001)
fig1 = plt.figure(figsize=(10, 10))
tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
plt.plot(tracks[0,:,0],tracks[0,:,1])
plt.plot(tracks[1,:,0],tracks[1,:,1])
plt.show()


Now the fun part is to see how well this scales. Let's try to simulate 1000 particles and comapre it with our two optimzied scenarios, lets see how we things look. 

In [ ]:
import time

np.random.seed(0)
nparts=10000
nsteps=1
X = np.random.random((nparts, 2)) * 2 - 1
V = np.random.random((nparts, 2)) * 0.1 

bodies = []
for i0 in range(len(X)):
    mass=1. # they will all have a mass of a solar mass
    pBody = body(X[i0],V[i0],mass)
    bodies.append(pBody)


t0=time.perf_counter()

treeStar = TreeStars(bodies,X)
lX=treeStar.allsteps(insteps=nsteps)

t1=time.perf_counter()
print("N-body Tree Delta time:",t1-t0)

#Now lets setup a quick init script only for up to 4 stars 
def initN(X,V,M,iN):
    pos=X
    mass=M#np.ones(iN)#units of Solar Mass
    #now v
    v=V
    #v=v.T
    #transform to the com frame
    #vcom=np.dot(mass,v)/np.sum(mass)
    #v-=vcom
    return pos,v,mass


def simTest(iX,iV,iM,iN=nparts,insteps=nsteps,dt=0.001):
    #units of years
    pos,vel,mass=initN(iX,iV,iM,iN)
    allstars = stars(mass,pos,vel,iN)
    allstars.parallelAcc()
    allstars.firststep(dt)
    for t in range(insteps):
        allstars.parallelAcc()
        allstars.step(dt)
    x1vals=np.reshape(allstars.posh,(insteps,iN,2))
    x1vals=np.swapaxes(x1vals,0,1)
    return x1vals

mass=np.ones(nparts)
t2=time.perf_counter()
simTest(X,V,mass,nparts)
t3=time.perf_counter()

print("N-body Parallel",t3-t2)

Now given all of this, lets try to do a realistic sampling and make a video of what motion might look like in the center of the galaxy. Let's sample from a gaussian distribution of stars. In turns out that the average distance between stars in the center of the galaxy is 1000 AU. So what we can do is sample a gaussian distribution with a width $\sigma$ such that 

$$
\bar{\Delta x_{i}} = \frac{1}{N} \sum_{j} |x_{i}-x_{j}| 
$$

In this case, what we want is the minimimum distance, which is just given by the minimum distance if we sampled a gaussian. Since, I am too lazy to do this calculation let just do it numerically. 

In [ ]:
x = np.array([])
y = np.array([])
yerr = np.array([])
for i0 in np.arange(1,10000,100):
    n=i0
    v0 = np.array([])
    ntoys=100
    for pToy in np.arange(ntoys):
        vals=np.abs(np.random.normal(0,1,n))
        v0  = np.append(v0,np.min(vals))
    x = np.append(x,n)
    y = np.append(y,v0.mean())
    yerr = np.append(yerr,v0.std())
    
plt.errorbar(x,y,yerr=yerr)
plt.yscale('log')
plt.show()

What this means is tha for simulation with 1000 particles, we should sample a gaussian with $\sigma=1000$, this will get us rougly the right spacing that we can then use to look at galactic flow over time. Let's go ahead and give this a try. Also, what we can do is set the velocity to zero, to see hwo it evolves. 

In [ ]:
####Warning this box takes a few minutes to run
nparts=1000
nsteps=5000

np.random.seed(1)
R     = np.random.normal(0,1,nparts)*nparts*1000
theta = np.random.uniform(0,2*np.pi,nparts)
X = np.vstack((R*np.cos(theta),R*np.sin(theta)))
X = X.T
V = np.zeros((nparts,2))
mass=np.abs(np.random.normal(2,0.5,nparts))
bodies = []
for i0 in range(len(X)):
    pBody = body(X[i0],V[i0],mass[i0])
    bodies.append(pBody)


t0=time.perf_counter()

treeStar = TreeStars(bodies,X)
treeStar.allsteps(insteps=nsteps,dt=1000.0)

lXVals=simTest(X,V,mass,nparts,nsteps,dt=2000.0)
t1=time.perf_counter()
print("N-body Tree Delta time:",t1-t0)

images=animate(lXVals,iN=nparts,xmin=-500000,xmax=500000,ymin=-500000,ymax=500000,stepsize=10)
imageio.mimsave('./orbit_n.gif', images, fps=10)
Image(open('orbit_n.gif','rb').read())


In [ ]:
tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
images=animate(tracks,iN=nparts,xmin=-1000000,xmax=1000000,ymin=-1000000,ymax=1000000,stepsize=10)
imageio.mimsave('./orbit_nt.gif', images, fps=10)
Image(open('orbit_nt.gif','rb').read())


In [ ]:
images=animate(lXVals,iN=nparts,xmin=-1000000,xmax=1000000,ymin=-1000000,ymax=1000000,stepsize=10)
imageio.mimsave('./orbit_n_v2.gif', images, fps=10)
Image(open('orbit_n_v2.gif','rb').read())

In [ ]:
images=animate(lXVals,iN=nparts,xmin=-5000000,xmax=5000000,ymin=-5000000,ymax=5000000,stepsize=10)
imageio.mimsave('./orbit_n_v3.gif', images, fps=10)
Image(open('orbit_n_v3.gif','rb').read())

In [ ]:
tracks=treeStar.history()
tracks=np.swapaxes(tracks,0,1)
images=animate(tracks,iN=nparts,xmin=-5000000,xmax=5000000,ymin=-5000000,ymax=5000000,stepsize=10)
imageio.mimsave('./orbit_nt_v4.gif', images, fps=10)
Image(open('orbit_nt_v4.gif','rb').read())


<a name='section_16_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L16.5 ML N-body Problem </h2>  


Now, we are going to construct a very simple machine learning approach to stellar simulation. While, this approach will not really allow us to go into detail, we will start to grasp a concept of where machine learning is relevant in a simulation scenario. 

To this, we first need to pull in our usual simulation code dictating stellar evolution. We will be sure to use the leapfrog symplectic integrator. Additionally, we will use a fixed time-step. 

Ok, so looks ok; could be better, but thats beyond this class. 

Now there is a broad range of possibilities that we can consier machine learning for. Ideally, we would want to use the fact that machine learning allows us to do a function approximation, but very fast and effectively. For the use of this in simulation, we can do just that, a funcitonal approximation. 

Now, lets consider the situation where we have randomly sample some initial conditions and then compute a tractory for that, then from that try to predict something. 

To randomly sample, we are going to have same over a restricted phase space. The issue is that a full random sample of many parametes is just hard to simulate, we are trying to predict a broad range of final states, this is tricky.

So for this sampling, what we will do is restrict our possible cases. We are going to restrict to a some simplified case where things are easy to solve. 


In [ ]:
##So now we are going to write a script that generates 1000 random conditions and trains a neural network to predict then ext 30 time steps
##We are going to do this for 2 body to start with
##To keep the dimensionality down to something reasonable, we are going to do this in the COM frame

def randomSamplePlanets(iN=2):
    ###Lets simplify the coordinates
    ###sample a radius along the x-axis 
    #radius=np.ones(iN)
    #radius[0]=-1    
    radius=np.random.uniform(0.5,2,iN)#units of AU
    radius[1]=-radius[0]
    #theta=np.random.uniform(0,2.*np.pi,iN)    
    #merge positions and make sure each row is planet
    theta=0
    pos=np.vstack((radius*np.cos(theta),radius*np.sin(theta)))
    pos=pos.T
    #now mass
    mass=np.random.uniform(0,3,iN)#units of Solar Mass
    #mass=np.ones(iN)#units of Solar Mass
    #now v
    #v=circle_v(mass,np.abs(radius))*np.random.normal(0,3)
    v=circle_v(mass,np.abs(radius))*np.abs(np.random.normal(0,1))
    theta=np.random.uniform(0,2.*np.pi,iN)
    v=np.vstack((v*np.cos(theta),v*np.sin(theta)))
    v=v.T
    #transform to the com frame
    vcom=np.dot(mass,v)/np.sum(mass)
    v-=vcom
    return pos,mass,v



Now what are going to do is try to predict the orbital positions of stars after running simulation for a large number of steps. To do this there are a few different things we can try to predict with the NN
 
 * The star trajectory in position space
 * The star position and momentum vector after many steps
 * The cgange in the position and the momentum vector 
 * The velocity change over time
 * The trajectory and the velocity. 
 
Let's run our simulator and set it up to output these possibilities. Additionally, now we will run 15k simulations so that we can we deep learn the many possible trajectories. 

In [ ]:
def sim(pos,mass,v,iN=2,dt=0.001,nsteps=5,iPlot=False,iOutput=0):
    #make the stars
    allstars = stars(mass,pos,v,iN)
    #simulate
    allstars.parallelAcc()
    allstars.totalE()
    allstars.firststep(dt)
    for t in range(nsteps):
        allstars.parallelAcc()
        allstars.totalE()
        allstars.step(dt)
    x1vals=np.reshape(allstars.posh,(nsteps,2*iN))
    vvals =np.reshape(allstars.velh,(nsteps,2*iN))
    if iPlot:
        for i0 in np.arange(iN):
            plt.plot(x1vals[:,2*i0],x1vals[:,2*i0+1])
        plt.show()

        plt.plot(range(nsteps),allstars.toth,label='Total')
        plt.legend()
        plt.xlabel('time step')
        plt.ylabel('energy')
        plt.show()
    if iOutput==0: #full trajectory
        return _,x1vals
    if iOutput==1: #output velocity and position
        return _,np.hstack((x1vals[-1],vvals[-1]))
    if iOutput==2: #differences from beginning
        dx1=x1vals[-1]-x1vals[0]
        dv1=vvals[-1]-vvals[0]
        return _,np.hstack((dx1,dv1))
    if iOutput==3:
        return _,vvals
    if iOutput==4:
        return x1vals,vvals

    
def makeTrainDataSet(iNPoints=15000,iN=2,iOutput=0):
    nsteps=50
    if iOutput == 4:
        indataset  = np.empty((0,4*nsteps), int)
    else: 
        indataset  = np.empty((0,10), int)
    if iOutput == 0 or iOutput == 3 or iOutput == 4:
        outdataset = np.empty((0,4*nsteps), float) 
    else:
        outdataset = np.empty((0,8), float) 
    for niter in range(iNPoints):
        pos,mass,v=randomSamplePlanets(iN)
        pIn,out=sim(pos,mass,v,nsteps=nsteps,iOutput=iOutput)
        outdataset= np.vstack((outdataset,out.flatten()))
        merge     = np.concatenate((v.flatten(),pos.flatten(),mass.flatten()))
        indataset = np.vstack((indataset,merge))
        #indataset = np.vstack((indataset,pIn.flatten()))
    return indataset,outdataset

pos,mass,v=randomSamplePlanets(2)
sim(pos,mass,v,nsteps=5000,iPlot=True)
trainset,targets=makeTrainDataSet()


Now our goal is to construct a deep learning model that allows us to predict the next 50 steps of our bath. Given our simulated dataset, we can go ahead and try to predict what is going on through a deep learning regression. 

Let's go ahead and make the model We will just make a simple ML model with a 4 layers and 20 hidden parameters

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable

class MLP(nn.Module):
    def __init__(self,n_inputs,n_outputs):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_inputs, 20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.ReLU(),
            nn.Linear(20, n_outputs),
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

def train(x,y,net,loss_func,opt,sched,nepochs):
    net.train(True)
    for epoch in range(nepochs):
        prediction = net(x)
        opt.zero_grad()
        loss = loss_func(prediction,y) 
        loss.backward() 
        opt.step()
        if epoch % 500 == 0: 
            print('[%d] loss: %.4f ' % (epoch + 1, loss.item()  ))
    sched.step()
    return

model     = MLP(trainset.shape[1],targets.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1, last_epoch=-1, verbose=False)
loss_fn   =  nn.MSELoss()

Finally, we can go ahead and train this guy

In [ ]:
tmp_tot    = torch.tensor(trainset).float()
tmp_label  = torch.tensor(targets).float()
train(tmp_tot,tmp_label,model,loss_fn,optimizer,scheduler,15001)

Now we can go ahead and run on a validation dataset and see how things are performing. 

In [ ]:
valid_trainset,valid_targets=makeTrainDataSet(iNPoints=10)
tmp_train  = torch.tensor(valid_trainset).float()
tmp_target = torch.tensor(valid_targets).float()
output_targets=model(tmp_train)
print(output_targets.shape)

#output_targets=output_targets.reshape(output_targets.shape[0],5,4).detach().numpy()
#valid_targets =tmp_target.reshape(tmp_target.shape[0],5,4).detach().numpy()
output_targets=output_targets.reshape(output_targets.shape[0],50,4).detach().numpy()
valid_targets =tmp_target.reshape(tmp_target.shape[0],50,4).detach().numpy()
#output_targets=output_targets.reshape(output_targets.shape[0],1,8).detach().numpy()
#valid_targets =tmp_target.reshape(tmp_target.shape[0],1,8).detach().numpy()

In [ ]:
for batch in np.arange(10):
    for i0 in np.arange(2):
        plt.plot(output_targets[batch,:,0+2*i0],output_targets[batch,:,1+2*i0], color='red',label='pred')
        plt.plot(valid_targets [batch,:,0+2*i0],valid_targets [batch,:,1+2*i0], color='blue',alpha=0.5,label='true')
    plt.legend()
    plt.show()

Ok, its nice that we can predict the paths quite well, and I find it pertty remarkable to be honest. However, to really make a power prediction, what we would like to do is predict the position and velocity after 50 steps. This way we can reduce the overall step size, and let the neural network do the "inpainting" of what is going on. 

In [ ]:
trainset,targets=makeTrainDataSet(iOutput=2)
model     = MLP(trainset.shape[1],targets.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1, last_epoch=-1, verbose=False)
loss_fn   =  nn.MSELoss()
tmp_tot    = torch.tensor(trainset).float()
tmp_label  = torch.tensor(targets).float()
train(tmp_tot,tmp_label,model,loss_fn,optimizer,scheduler,15001)

In [ ]:
valid_trainset,valid_targets=makeTrainDataSet(iNPoints=10,iOutput=2)
tmp_train  = torch.tensor(valid_trainset).float()
tmp_target = torch.tensor(valid_targets).float()
output_targets=model(tmp_train)
output_targets=output_targets.reshape(output_targets.shape[0],1,8).detach().numpy()
valid_targets =tmp_target.reshape(tmp_target.shape[0],1,8).detach().numpy()
#print(valid_targets.shape)
for batch in np.arange(10):
    for i0 in np.arange(2):
        plt.arrow(output_targets[batch,0,0+2*i0],output_targets[batch,0,1+2*i0],output_targets[batch,0,4+2*i0],output_targets[batch,0,5+2*i0],width=.05, edgecolor='red',facecolor='red')
        plt.arrow(valid_targets [batch,0,0+2*i0],valid_targets [batch,0,1+2*i0],valid_targets [batch,0,4+2*i0],valid_targets [batch,0,5+2*i0],width=.05, edgecolor='blue',facecolor='blue',alpha=0.5)
    plt.show()

Now, what we can do is try to improve our simulation by requiring that our system conserves energy. The simplest way to do that is to add energy conservation as a constraint in the loss function. Since we are computing the the change in the relative position of the planets, what this means is that the derivative of the total energy is zero. We can write this as: 

$$
\mathcal{H} = \frac{1}{2} m_{1} v_{1}^{2} + \frac{1}{2} m_{2} v_{2}^2 + \frac{Gm_{1}m_{2}}{|\vec{r}_{1}-\vec{r}_{2}|}\\
\frac{d\mathcal{H}}{dt} = 0 \\
\frac{d\mathcal{H}}{dt}  = mv_{1} \frac{dv_{1}}{dt} + mv_{2} \frac{dv_{2}}{dt} + \frac{Gm_{1}m_{2}}{|\vec{r}_{1}-\vec{r}_{2}|^{2}}\left(\frac{dr_{1}}{dt} + \frac{dr_2}{dt}\right) = 0
$$

Additionally, we can could also impose angular momentum conservation

$$
L_{\rm tot} = m_{1} v_{1} r_{1} +  m_{2} v_{2} r_{2}\\
\frac{d L_{\rm tot}}{dt} = m_{1}\left(\frac{dv_{1}}{dt} r_{1} + v_{1}\frac{dr_{1}}{dt}\right) + m_{2}\left(\frac{dv_{2}}{dt} r_{2} + v_{2}\frac{dr_{2}}{dt}\right)
$$

Mostly in the inerest of being lazy, lets add angular momentum conservation conservation as a constraint. 

In [ ]:
def energy(iVec):
    dr = torch.pow(torch.norm(iVec[:,4:6]-iVec[:,6:8],dim=1),-0.5)
    v2 = torch.sum(iVec[:,0:4]**2,dim=1)
    return dr+v2

def newEloss(prediction,y_data,iloss=loss_fn,lam=1e-1):
    #rint(prediction.shape,y_data.shape)
    mse=iloss(prediction,y_data)
    en1=energy(y_data)
    en0=energy(prediction)
    #grad=torch.autograd.grad(en0, inputs, torch.ones_like(en0), create_graph=True)[0]
    return mse + lam*torch.mean((en1-en0)**2)

def ang(iInput,iPred):
    im1 = iInput[:,8]
    im2 = iInput[:,9]
    iv1 = iInput[:,0:2]
    iv2 = iInput[:,2:4]
    ir1 = iInput[:,4:6]
    ir2 = iInput[:,6:8]
    dr1 = iPred[:,0:2]
    dr2 = iPred[:,2:4]
    dv1 = iPred[:,4:6]
    dv2 = iPred[:,6:8]
    return im1*(torch.sum(dv1*ir1,dim=1)+torch.sum(iv1*dr1,dim=1))+im2*(torch.sum(dv2*ir2,dim=1)+torch.sum(iv2*dr2,dim=1))
    
def newAloss(prediction,y_data,iInput,iloss=loss_fn,lam=1e-1):
    dL=ang(iInput,prediction)
    mse=iloss(prediction,y_data)
    return mse + lam*torch.mean(dL**2)
    
valid_trainset,valid_targets=makeTrainDataSet(iNPoints=10,iOutput=1)
tmp_train  = torch.tensor(valid_trainset).float()
tmp_target = torch.tensor(valid_targets).float()
output_targets=model(tmp_train)

def trainA(x,y,net,loss_func,opt,sched,nepochs):
    net.train(True)
    for epoch in range(nepochs):
        prediction = net(x)
        opt.zero_grad()
        loss = loss_func(prediction,y,x) 
        loss.backward() 
        opt.step()
        if epoch % 500 == 0: 
            print('[%d] loss: %.4f ' % (epoch + 1, loss.item()  ))
    sched.step()
    return

losstest = newAloss(output_targets,tmp_target,tmp_train,loss_fn)
print(losstest,losstest.backward())

In [ ]:
trainA(tmp_tot,tmp_label,model,newAloss,optimizer,scheduler,5001)

In [ ]:
valid_trainset,valid_targets=makeTrainDataSet(iNPoints=10,iOutput=2)
tmp_train  = torch.tensor(valid_trainset).float()
tmp_target = torch.tensor(valid_targets).float()
output_targets=model(tmp_train)
output_targets=output_targets.reshape(output_targets.shape[0],1,8).detach().numpy()
valid_targets =tmp_target.reshape(tmp_target.shape[0],1,8).detach().numpy()
#print(valid_targets.shape)
for batch in np.arange(10):
    for i0 in np.arange(2):
        plt.arrow(output_targets[batch,0,0+2*i0],output_targets[batch,0,1+2*i0],output_targets[batch,0,4+2*i0],output_targets[batch,0,5+2*i0],width=.05, edgecolor='red',facecolor='red')
        plt.arrow(valid_targets [batch,0,0+2*i0],valid_targets [batch,0,1+2*i0],valid_targets [batch,0,4+2*i0],valid_targets [batch,0,5+2*i0],width=.05, edgecolor='blue',facecolor='blue',alpha=0.5)
    plt.show()

In [ ]:
def prepInputs(iPos,iV,iM):
    dvec=iPos[0]-iPos[1]
    theta = -1.*np.arctan2(dvec[1],dvec[0])
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    #rotate
    opos=np.reshape(np.zeros(4),(2,2))
    opos[0] = np.dot(rot, iPos[0])
    opos[1] = np.dot(rot, iPos[1])
    ovel=np.reshape(np.zeros(4),(2,2))
    ovel[0] = np.dot(rot, iV[0])
    ovel[1] = np.dot(rot, iV[1])
    #set y-coordinate to exactly 0
    #opos[0,1]=0
    #opos[1,1]=0
    merge=np.concatenate((ovel.flatten(),opos.flatten(),iM.flatten()))
    lInput=torch.tensor(merge).float()
    return lInput


#def prepOutput(iPos,iInf):
#    theta = -1.*np.arctan2(iPos[:,1],iPos[:,0])
#    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
#    #rotate
#    lPos   = iInf
#    opos=np.reshape(np.zeros(4),(2,2))
#    opos[0] = np.dot(rot, lPos[0])
#    opos[1] = np.dot(rot, lPos[1])
#    return opos

#Now lets run a simulator for this 
def mlSim(pos,mass,v,iN=2,dt=0.001,nsteps=10,iPlot=False,iOutput=1):
    posh = np.array([])
    velh = np.array([])
    lPos = pos
    lV   = v
    lInput = prepInputs(pos,v,mass)    
    for i0 in range(nsteps):
        output = model(lInput)
        lPos   += output[0:4].reshape(2,2).detach().numpy()
        lV     += output[4:8].reshape(2,2).detach().numpy()
        posh   = np.append(posh,lPos)
        velh   = np.append(velh,lV)
        lInput = prepInputs(lPos,lV,mass)
    if iPlot:
        posh=np.reshape(posh,(nsteps,2*iN))
        for i0 in np.arange(iN):
            plt.plot(posh[:,2*i0],posh[:,2*i0+1])
        plt.show()

np.random.seed(10)
pos,mass,v=randomSamplePlanets(2)
import time
start_time = time.time()
sim(pos,mass,v,nsteps=500,iPlot=True)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
mlSim(pos,mass,v,iPlot=True,nsteps=10)
print("--- %s seconds ---" % (time.time() - start_time))


<a name='section_16_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L16.6 1D Hydrodynamical Solutions </h2>  



Note: from here https://zingale.github.io/computational_astrophysics/ODEs/ODEs-conservation.html

Now that we have developed the concept of how to solve 1D differential equations. Moreover, we have even applied this to the notion of planetary interations. We would like to do consider a regular grid of objects, kind of like what we did with the tree above. However, now we are going to keep our objects fixed, and use the fact that they are fixed to try to solve a partial differential equation. 

First, lets imagine we have a box filled with water, and we discritzie this box to have various levels of water. 

In [ ]:
import scipy.stats as stats

ngrid=10
gridx=np.arange(-10,10,20/ngrid)
gridy=stats.norm.pdf(gridx,0,3)
plt.plot(gridx, gridy, drawstyle='steps-mid', label='steps-mid')
plt.xlabel('x-position')
plt.ylabel('height')
plt.show()

Now imagine for some crazy reason, that this a mountain of water describd by a discretized probability distribution function given by above, and this mountain is moving with some velocity $v$. The equation of motion for this moutain can be written as 

$$
\frac{\partial p}{\partial x} - v \frac{\partial p}{\partial t} = 0
$$

This is often called the 1D Euler equation or just the Burger's equation. The solution to this is the pdf $p(x,t)$ moves with a velocity of $v$ in this case its a normal distribution sliding to the right. 

$$
p(x,t) = \mathcal{N} (\mu=x-vt,\sigma)
$$

We can also solve this numerically by noting that we can translate each element of our grid into motion characterized by its neighbour. The way we do this is tha for each bin ($b^{t+1}_{i}$) at time $t+1$ we modify it during time by its neighbours and its previous bin by noting the motion of the system will cause the current bin to lose a fraction of its elements with the fraction given by $C=\frac{\Delta x_{t+1-t}}{\Delta x_{\rm bin}}$ the amount of motion in a step over the bin size, which we can writing noting the velocity $v$ as 

$$
b^{t+1}_{i} = b^{t}_{i} - \frac{v\Delta t}{\Delta x} b^{t}_{i} + \frac{v\Delta t}{\Delta x} b^{t}_{i-1}
$$

where the last term is the addition of stuff coming from the bin right next to it. As a side note, the term $C=\frac{\Delta x_{t+1-t}}{\Delta x_{\rm bin}}$ is called the Courant-Fredrichs-Lewy condition. 

Ok now lets evolve it, to make things easier, we are going to do this with video game geomtery, or if you jump on the right you will end up on the left. For this, we are going to use the roll command, lets quickly see how it works. 


In [ ]:
array=np.arange(1,10,1)
print(array)
print(np.roll(array,1))

Our strategy will then be to shift our distribution by one unit, and then average the current distribution with the shifted distribution by the velocity. 

In [ ]:
def evolvepdf(xvals,pdf,v,dt):
    C = v*dt/(xvals[1]-xvals[0])
    pdf = pdf*(1-C) + np.roll(pdf,-1)*C 
    #pdf = pdf + np.roll(pdf,-1)*C*0.5 - np.roll(pdf,1)*C*0.5 
    return pdf

#ok lets step it once and check
gridx=np.arange(-10,10,20/ngrid)
gridy=stats.norm.pdf(gridx,0,3)
plt.plot(gridx,gridy,drawstyle='steps-mid')
newgridy=gridy
for i0 in range(1):
    newgridy=evolvepdf(gridx,newgridy,1,1)
plt.plot(gridx,newgridy,drawstyle='steps-mid')
plt.show()

#now lets step it a few times and make a video
    


Now the interesting thing is to test this guy over time. What if we evolve a top hat distribution moving to the right 100 steps or even more? Ideally, our top-hat should be preserved, but this shifting is definitely not perfect. Lets go ahead and evolve our top hat 2000 steps over a distance of 20. This should just reproduce the original top hat distribution at the same position. What do you think will happen? 

In [ ]:
#now lets evolve a hat with 100 steps
ngrid=100
gridx=np.arange(-10,10,20/ngrid)
gridy=np.ones(gridx.shape)
gridy[0:20]=0
gridy[80:100]=0
#gridy=np.sin(gridx)**2
gridy*=1./np.sum(gridy) #normalize it

def evolve(igridx,igridy,idt,iv):
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    niter=2000
    for i in range(niter):
        igridy=evolvepdf(igridx,igridy,iv,idt)
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    return igridy
    
outy=evolve(gridx,gridy,0.1,0.1) #if we do 1000 iterations of 0.1 that is a distance of 100 or 10 times around


You can see that the top hat is starting to deflate over time. This deflation is a result of the inaccuracy of our sysem to evolve the fluid flow. It eventually starts to fall apart, this is the issue with numerical simulations that we have seen time and time again. Numerics are only so good without some physics intution. 


A quick way to understand why the convergence breaks down, we can imagine fourier transforming our tophat distribution. If we consider a single fourier mode our amplitude formula. Let's take our amplitude formula

$$
b^{t+1}_{i} = b^{t}_{i} - \frac{v\Delta t}{\Delta x} b^{t}_{i} + \frac{v\Delta t}{\Delta x} b^{t}_{i-1} \\
b^{t+1}_{i} = b^{t}_{i} - \frac{C}{2} \left( b^{t}_{i} -  b^{t}_{i-1} \right)
$$

Now isolating a single Fouier mode we have with $I=\sqrt{-1}$ to avoid complications in toation

$$
\tilde{b}^{t+1}_{i}e^{Ii\theta} = \tilde{b}^{t}e^{Ii\theta} - \frac{C}{2} \left(  \tilde{b}^{t}e^{I(i+1)\theta}  -  \tilde{b}^{t} e^{I(i-1)\theta}  \right) \\
\tilde{b}^{t+1}_{i} = \tilde{b}^{t}\left(1 - \frac{C}{2} \left(  e^{I\theta}  -   e^{I\theta}  \right)\right) \\
\tilde{b}^{t+1}_{i} = \tilde{b}^{t}\left(1 - C I\sin\theta\right)\\
$$ 

or in other words the change in the amplitude of this mode is given by 

$$
|\frac{\tilde{b}^{t+1}}{\tilde{b}^{t}}|^{2} = 1+C^2\sin^{2}\theta
$$

This means that as long as we continue to iterate over and over again, we will always smear things out (adding mroe and more fouier modes). Eventually we will just end up with something flat. 

Lets now try construct something that is a little more robust. Lets consider our distribution is an approximation of a a continuous distribution $f(x)$. We can thus make a computation of the derivative of the pdf by noting that in a histogram the best esitmate of the pdf is from the center of the bin. This gives us

$$
\frac{df}{dx} = \frac{f(x+\frac{\Delta x}{2})-f(x-\frac{\Delta x}{2})}{\Delta x}
$$

As a result, we can then write

$$
f(x,t+\Delta t) = f(x) + \frac{\Delta x}{\Delta t} \Delta t  \frac{df}{dx}
$$

and a better approximation becomes 

$$
f(x,t+\Delta t) = f(x) + \frac{\Delta x}{\Delta t} \Delta t  \frac{df}{dx}(t+\frac{\Delta t}{2})
$$


In [ ]:
def dpdfdx(xvals,pdf):
    dx=(xvals[1]-xvals[0])
    dpdfdx     = 0.5*(np.roll(pdf,2)-pdf)#df/dx (dx=bin)
    pdfshift = pdf    + 0.5*dpdfdx#f(x+dx/2) => shift by 1/2 a bin
    #nwo the diervaitve
    return (np.roll(pdfshift,-1)-pdfshift)/dx #df(x+dx/2)/dx

def evolvepdf(xvals,pdf,v,dt):
    C = v*dt
    dpdf     = dpdfdx(xvals,pdf)
    pdf2     = pdf + 0.5*C*dpdf #f(x,t+dt/2)
    dpdf     = dpdfdx(xvals,pdf2)  #dff(x,t+dt/2)/dx
    pdf      += C*dpdf #shift it
    return pdf


def evolve(igridx,igridy,idt,iv):
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    niter=2000
    for i in range(niter):
        igridy=evolvepdf(igridx,igridy,iv,idt)
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    return igridy

#now lets evolve a hat with 100 steps
ngrid=100
gridx=np.arange(-10,10,20/ngrid)
gridy=np.ones(gridx.shape)
gridy[0:20]=0
gridy[80:100]=0
#gridy=np.sin(gridx)**2
outy=evolve(gridx,gridy,0.1,0.05)
print(np.sum(gridy),np.sum(outy))


Now, we can do even better than this by improving our approximations This is very much like what we did with the leapfrog method and the Runge-kutta methods. Here what we are doing is improving our estimation of the derivative. We can do this by 

$$
\frac{df}{dx} = \min\left(\frac{f(x)-f(x-\Delta x)}{\Delta x} , \frac{f(x+\Delta x)-f(x)}{\Delta x}\right) \\
$$

Now noting that derivate if there is a minima in a bin ie 


$$
\frac{f(x)-f(x-\Delta x)}{\Delta x} \times \frac{f(x+\Delta x)-f(x)}{\Delta x} < 0
$$

we take a slope of zero. From there we can then go ahead and compute

$$
\frac{df}{dx}(x+\frac{\Delta x}{2}) 
$$

using the above distribution. 

In [ ]:
def dpdfdx(xvals,pdf):
    dx=(xvals[1]-xvals[0])
    dpdfu     = (np.roll(pdf,1)-pdf)#b_[i+1]-b_{i}
    dpdfd     = (pdf-np.roll(pdf,-1))#b_[i]-b_{i-1}
    dpdfmin   = np.where(np.fabs(dpdfd) < np.fabs(dpdfu), dpdfd, dpdfu) #min diff
    dpdfmin   = np.where(dpdfd*dpdfu > 0, dpdfmin, 0) #min diff
    pdfshift  = pdf    + 0.5*dpdfmin#shift by 1/2 a bin
    return (np.roll(pdfshift,-1)-pdfshift)/dx #This ensures integral is conserved

def evolvepdf(xvals,pdf,v,dt):
    C = v*dt
    dpdf     = dpdfdx(xvals,pdf)
    pdf2     = pdf + 0.5*C*dpdf #shift it half
    dpdf     = dpdfdx(xvals,pdf2)  #dff(x,t+dt/2)/dx
    pdf      += C*dpdf #shift it
    return pdf

def evolve(igridx,igridy,idt,iv):
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    niter=2000
    for i in range(niter):
        igridy=evolvepdf(igridx,igridy,iv,idt)
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    return igridy

#now lets evolve a hat with 100 steps
ngrid=100
gridx=np.arange(-10,10,20/ngrid)
gridy=np.ones(gridx.shape)
gridy[0:20]=0
gridy[80:100]=0
#gridy=np.sin(gridx)**2
outy=evolve(gridx,gridy,0.1,0.05)
print(np.sum(gridy),np.sum(outy))

ok now lets actually do the inviscit Berger's equation  ( https://en.wikipedia.org/wiki/Burgers%27_equation) 

$$
\frac{\partial v}{\partial t} - v \frac{\partial v}{\partial x} = 0 \\
\frac{\partial v}{\partial t} - \frac{1}{2} \frac{\partial v^{2}}{\partial x}=0
$$

This shows the diffusion flow a system. The difference with above is that the $v$ is now the funciton we want to solve. 

Instead of a pdf, we are now solving for $v(x,t)$. 

The way we are going to do this is we now need to compute 

$$
\frac{\partial u}{\partial x}  = \min\left(\frac{u(x)-u(x-\Delta x)}{\Delta x} , \frac{u(x+\Delta x)-u(x)}{\Delta x}\right)
$$

Then what we do is approximate is the velocity going out of the bin, and the velocity of the fluid going into the bin

$$
u_{l} = u(x-\frac{\Delta x}{2})\\
u_{r} = u(x+\frac{\Delta x}{2})
$$

If we have $u_{r} > u_{l}\rightarrow u_{r} + u_{l} > 0$  then we have that fluid is leaving the bin. However if its less than 0, then we have that fluid is piling up.  At this stage 


In [ ]:
def dpdfdx(xvals,pdf,shock=False):
    dx=(xvals[1]-xvals[0])
    #compute
    dpdfr     = (np.roll(pdf,-1)-pdf)#b_[i+1]-b_{i}
    dpdfl     = (pdf-np.roll(pdf,1))#b_[i]-b_{i-1}
    dpdfmin   = np.where(np.fabs(dpdfl) < np.fabs(dpdfr), dpdfl, dpdfr) #min diff
    dpdfmin   = np.where(dpdfr*dpdfl > 0, dpdfmin, 0) #if sign change take zero
    #Now compute the difference
    pdfshiftl = pdf    - 0.5*dpdfmin#f(x-dX/2)
    pdfshiftr = np.roll(pdf,1)    + 0.5*np.roll(dpdfmin,1)#f(x+dx/2) => note shift to match the two
    
    #shock wave
    S = 0.5 * (pdfshiftr + pdfshiftl)#aggregate velocity  
    ushock = np.where(S   > 0.0, pdfshiftr, pdfshiftl) #computate aggregate velocity take ther right direction given
    ushock = np.where(S  == 0.0, 0.0, ushock) #Check its not zero

    #normal wave
    urare  = np.where(pdfshiftl <= 0.0, pdfshiftl, 0.0) #if right vel < 0 keep it
    urare  = np.where(pdfshiftr >= 0.0, pdfshiftr, urare) #if left vel > 0 keep that over previous
    if shock:
        us     = np.where(pdfshiftr > pdfshiftl, ushock, urare) #if right is greater than left (compression) do shock
    else:
        us     = urare
    #now shift the bin
    return (np.roll(us,0)**2 - np.roll(us,-1)**2)/dx

def evolvepdf(xvals,pdf,dtstep,shock=False):
    eps=0.1
    dx=(xvals[1]-xvals[0])
    dt = dtstep*dx/(np.max(np.abs(pdf))+eps)
    C = 0.5*dt
    dpdf     =  dpdfdx(xvals,pdf,shock)
    pdf2     =  pdf + 0.5*C*dpdf #shift it half
    dpdf     =  dpdfdx(xvals,pdf2,shock) #compute exptected shift
    pdf      += C*dpdf #shift it
    return pdf,dt


def makePlot(ixvals,iyvals,ax,fig,images,ymin=-1,ymax=1,xmin=0,xmax=1.0):
    # plot and show learning process
    plt.cla()
    ax.set_xlabel('x', fontsize=24)
    ax.set_ylabel('v(x)', fontsize=24)
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    ax.plot(ixvals,iyvals,drawstyle='steps-mid')
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)


def evolve(images,igridx,igridy,idt,shock=False):
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    niter=250
    time=0
    fig, ax = plt.subplots(figsize=(12,7))
    for i in range(niter):
        igridy,pdt=evolvepdf(igridx,igridy,idt,shock=shock)
        time+=pdt
        makePlot(igridx,igridy,ax,fig,images)
    plt.plot(igridx,igridy,drawstyle='steps-mid')
    return igridy

#now lets evolve a hat with 100 steps
ngrid=100
gridx=np.arange(0,1,1/ngrid)
gridybase=np.zeros(gridx.shape)
def sine(igridx,igridy):
    index = np.logical_and(igridx >= 0.333,igridx <= 0.666)
    igridy[index] += gridx[index]*(1.5*np.sin(2.0*np.pi*(igridx[index]-0.333)/0.333))
    return igridy
gridy=sine(gridx,gridybase)

gridy=stats.norm.pdf(gridx,0.5,0.2)-1
#gridy[0:10]=0
#gridy[90:100]=0

#gridy=np.ones(gridx.shape)*0.8
#gridy[0:10]=0
#gridy[90:100]=0

images=[]
outy=evolve(images,gridx,gridy,0.5,shock=True)

from IPython.display import Image
imageio.mimsave('./berger.gif', images, fps=10)
Image(open('berger.gif','rb').read())


#gridybase=np.zeros(gridx.shape)
#gridy=sine(gridx,gridybase)
#outy=evolve(gridx,gridy,0.5,shock=False)
#plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-16.1.1 </span>

Repeat the top hat evolution with 1/10th the time step, how much does this lead to the diulation? 




In [ ]:

(Question for later, how does the time step and velocity impact our precisiion? )
Q: What happens for a sine wave? 


<a name='section_16_7'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L16.7 N-D Hydrodynamical Solutions </h2>  


A multi-dimensional version of the above equations starts from the Euler equation. We can write the Euler equation for conservation of mass 

$$
\frac{\partial \rho}{\partial t} + \nabla\cdot (\rho \vec{u}) = 0
$$

Next, we can write the Euler equation for conservation of momentum as

$$
\frac{\partial \rho \vec{u} }{\partial t} + \nabla\cdot (\rho \vec{u} \otimes \vec{u} + \vec{P} ) = -\rho\nabla\Phi
$$

where the term on the right is just the external force. 

Finally, we can write the Euler equation for conservation of Energy as

$$
\frac{\partial \rho E }{\partial t} + \nabla\cdot (\rho E \vec{u} + p\vec{u} ) = 0
$$

Often they are written in vector form 

$$
\frac{\partial q_{i}}{\partial t} + \frac{\partial f}{\partial q_{i}} = 0
$$

where we have 

$$
f 
$$

For $f$ one the of the functions above. The above set of equations, thus constitutes a set of coupled differential equations that we need to solve, and are in all practial purposes a generalization of the equations above. In light of this we are going to do something more complicated. 

Let's take our middle equation, and rewrite it slightly in a simpler form. We can write this as

$$
\frac{d\vec{v}}{dt} = -\frac{1}{\rho}\nabla \vec{P} - \frac{1}{\rho}\vec{f}
$$

If we consider a spherically symetric setup, then we can write the pressure $P(r)=k\rho^{1+1/n}$ essentially, we are just saying it scales with $\rho$. Additionally, we can write the force $f(r)=-\lambda r - \nu v$ in terms of the gravitational pressurem $\lambda$ and the viscosity $\nu$

Furthermore, if we treat each point a spherically symmetric Gaussian, we can write

$$
\rho_{i}(r) = \left(\frac{1}{\sigma\sqrt{\pi}}\right)^{3} e^{-\frac{r_{i}^2}{\sigma^2}}
$$

The density at a specific point can then just given by the sum over all particles at that point

$$
\rho(\vec{r}) =  \sum_{i} m_{i} \rho_{i}\left(\vec{r}-\vec{r_{i}}\right)
$$
More over, with this discretiation, we can write
$$
\nabla \rho(\vec{r}) =  \sum_{i} m_{i} \nabla \rho_{i}\left(\vec{r}-\vec{r_{i}}\right)
$$

We can also write 
$$
\frac{1}{\rho}\nabla P = \nabla\left(\frac{P}{\rho}\right)+\frac{P}{\rho^2}\nabla\rho
$$

Noting 
$$
\frac{P}{\rho}(\vec{r_{i}}) = \sum_{j} \frac{P_{j}}{\rho^{2}_{j}} m_{j} \rho_{j} \left(\vec{r_{i}}-\vec{r_{j}}\right)\\
\nabla\left(\frac{P_{i}}{\rho}\right) = \sum_{j} \frac{P_{j}}{\rho^{2}_{j}} m_{j} \nabla \rho_{j} \left(\vec{r_{i}}-\vec{r_{j}}\right)
$$

Now combining this we have
$$
\frac{1}{\rho}\nabla P = \sum_{j\neq i} \left(\frac{P_{i}}{\rho^{2}_{i}} + \frac{P_{j}}{\rho^{2}_{j}}\right)\nabla \rho_{j} \left(\vec{r_{i}}-\vec{r_{j}}\right)
$$

The resulting acceleration is thus

$$
\frac{d\vec{v}}{dt} = -\sum_{j\neq i} \left(\frac{P_{i}}{\rho^{2}_{i}} + \frac{P_{j}}{\rho^{2}_{j}}\right)\nabla \rho_{j} \left(\vec{r_{i}}-\vec{r_{j}}\right) + \vec{f}
$$


More details here
http://www.astro.yale.edu/vdbosch/Numerical_Hydrodynamics.pdf
https://philip-mocz.medium.com/create-your-own-smoothed-particle-hydrodynamics-simulation-with-python-76e1cec505f1

In [ ]:
from scipy.special import gamma

def rhoP( x, y, z, h ):
    r = np.sqrt(x**2 + y**2 + z**2)  
    w = (1.0 / (h*np.sqrt(np.pi)))**3 * np.exp( -r**2 / h**2)
    return w

def gradrhoP( x, y, z, h ):
    r = np.sqrt(x**2 + y**2 + z**2)
    n = -2 * np.exp( -r**2 / h**2) / h**5 / (np.pi)**(3/2)
    wx = n * x
    wy = n * y
    wz = n * z
    return wx, wy, wz

def getPairwiseSeparations( ri, rj ):
    M = ri.shape[0]
    N = rj.shape[0]
  
    # positions ri = (x,y,z)
    rix = ri[:,0].reshape((M,1))
    riy = ri[:,1].reshape((M,1))
    riz = ri[:,2].reshape((M,1))
  
    # other set of points positions rj = (x,y,z)
    rjx = rj[:,0].reshape((N,1))
    rjy = rj[:,1].reshape((N,1))
    rjz = rj[:,2].reshape((N,1))
  
    # matrices that store all pairwise particle separations: r_i - r_j
    dx = rix - rjx.T
    dy = riy - rjy.T
    dz = riz - rjz.T
    
    return dx, dy, dz

def getDensity( r, pos, m, h ):
    M = r.shape[0]
    dx, dy, dz = getPairwiseSeparations( r, pos );
    rho = np.sum( m * rhoP(dx, dy, dz, h), 1 ).reshape((M,1))
    return rho

def getPressure(rho, k, n):
    P = k * rho**(1+1/n)
    return P

def getAcc( pos, vel, m, h, k, n, lmbda, nu ):
    N = pos.shape[0]
    rho = getDensity( pos, pos, m, h )
  
    # Get the pressures
    P = getPressure(rho, k, n)
  
    # Get pairwise distances and gradients
    dx, dy, dz = getPairwiseSeparations( pos, pos )
    dWx, dWy, dWz = gradrhoP( dx, dy, dz, h )
  
    # Add Pressure contribution to accelerations   
    tmp1 = P/rho**2 
    tmp2 =  P.T/rho.T**2 
    tmp  = ( P/rho**2 + P.T/rho.T**2  )
    ax = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWx, 1).reshape((N,1))
    ay = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWy, 1).reshape((N,1))
    az = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWz, 1).reshape((N,1))
  
    # pack together the acceleration components
    a = np.hstack((ax,ay,az))
  
    # Add external potential force and viscosity
    a += -lmbda * pos - nu * vel
  
    return a

def makePlot(pos,ax1,ax2,fig,rho,images,rr,rlin,rho_analytic, m, h):
    plt.sca(ax1)
    plt.cla()
    cval = np.minimum((rho-3)/3,1).flatten()
    ax1.scatter(pos[:,0],pos[:,1], c=cval, cmap=plt.cm.autumn, s=10, alpha=0.5)
    ax1.set(xlim=(-1.4, 1.4), ylim=(-1.2, 1.2))
    ax1.set_aspect('equal', 'box')
    ax1.set_xticks([-1,0,1])
    ax1.set_yticks([-1,0,1])
    ax1.set_facecolor('black')
    ax1.set_facecolor((.1,.1,.1))

    plt.sca(ax2)
    plt.cla()
    ax2.set(xlim=(0, 1), ylim=(0, 3))
    ax2.set_aspect(0.1)
    ax2.plot(rlin, rho_analytic, color='gray', linewidth=2)
    rho_radial = getDensity( rr, pos, m, h )
    ax2.plot(rlin, rho_radial, color='blue')
    
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
            
def main(plotRealTime = True ):
    # Simulation parameters
    N         = 400    # Number of particles
    t         = 0      # current time of the simulation
    tEnd      = 12     # time at which simulation ends
    dt        = 0.04   # timestep
    M         = 2      # star mass
    R         = 0.75   # star radius
    h         = 0.1    # smoothing length
    k         = 0.1    # equation of state constant
    n         = 1      # polytropic index
    nu        = 1      # damping
    
    # Generate Initial Conditions
    np.random.seed(42)            # set the random number generator seed
    
    lmbda = 2*k*(1+n)*np.pi**(-3/(2*n)) * (M*gamma(5/2+n)/R**3/gamma(1+n))**(1/n) / R**2  # ~ 2.01
    m     = M/N                    # single particle mass
    pos   = np.random.randn(N,3)   # randomly selected positions and velocities
    vel   = np.zeros(pos.shape)

    # calculate initial gravitational accelerations
    acc = getAcc( pos, vel, m, h, k, n, lmbda, nu )

    # number of timesteps
    Nt = int(np.ceil(tEnd/dt))

    # prep figure
    fig, ax1 = plt.subplots(figsize=(12,7)) 
    grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
    ax1 = plt.subplot(grid[0:2,0])
    ax2 = plt.subplot(grid[2,0])
    rr = np.zeros((100,3))
    rlin = np.linspace(0,1,100)
    rr[:,0] =rlin
    rho_analytic = lmbda/(4*k) * (R**2 - rlin**2)
    images=[]

    # Simulation Main Loop
    for i in range(Nt):
        vel += acc * dt/2
        pos += vel * dt
        # update accelerations
        acc = getAcc( pos, vel, m, h, k, n, lmbda, nu )
        vel += acc * dt/2
        # update timef
        t += dt
        # get density for plottiny
        rho = getDensity( pos, pos, m, h )

        # plot in real time
        if plotRealTime or (i == Nt-1):
            makePlot(pos,ax1,ax2,fig, rho, images,rr,rlin,rho_analytic,m,h)

    # add labels/legend
    plt.sca(ax2)
    plt.xlabel('radius')
    plt.ylabel('density')
    
    # Save figure
    plt.savefig('sph.png',dpi=240)
    plt.show()
    return images

images=main()
imageio.mimsave('./xstar.gif', images, fps=10)
Image(open('xstar.gif','rb').read())


In [ ]:
Image(open('star.gif','rb').read())